In [35]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib as plt

# acled_df = pd.read_csv('../data/acled/darfur.csv') ## 9316 entries, 32 cols
# print(acled_df.info()) 
# print(acled_df.columns)
# acled_df['FATALITIES']


ucdp_df = pd.read_csv("../data/ucdp/darfur_113023.csv").drop('low_est', axis=1) ## 1669 entries, 48 cols
# print(ucdp_df.info()) 
print(ucdp_df.columns)
ucdp_df.dropna(inplace=True)
ucdp_df


Index(['id', 'relid', 'year', 'active_year', 'code_status', 'type_of_violence',
       'conflict_dset_id', 'conflict_new_id', 'conflict_name', 'dyad_dset_id',
       'dyad_new_id', 'dyad_name', 'side_a_dset_id', 'side_a_new_id', 'side_a',
       'side_b_dset_id', 'side_b_new_id', 'side_b', 'number_of_sources',
       'source_article', 'source_office', 'source_date', 'source_headline',
       'source_original', 'where_prec', 'where_coordinates',
       'where_description', 'adm_1', 'adm_2', 'latitude', 'longitude',
       'geom_wkt', 'priogrid_gid', 'country', 'country_id', 'region',
       'event_clarity', 'date_prec', 'date_start', 'date_end', 'deaths_a',
       'deaths_b', 'deaths_civilians', 'deaths_unknown', 'best_est',
       'high_est', 'COORDS'],
      dtype='object')


,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,dyad_dset_id,...,date_prec,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,best_est,high_est,COORDS
17,SUD-2015-1-833-105,2015,True,Clear,1,309,309,Sudan: Government,11988,11988,...,10/07/2015 00:00:00,10/07/2015 00:00:00,0,0,2,0,2,2,2,POINT (24.647945 13.287452)
64,SUD-2015-2-111-0,2015,True,Clear,2,5276,4666,Ma'aliyah - Rizeigat Baggara,5276,5276,...,05/10/2015 00:00:00,05/11/2015 00:00:00,85,130,0,0,215,215,134,POINT (26.5333333 11.5833333)
65,SUD-2017-2-5276-2.1,2017,True,Clear,2,5276,4666,Ma'aliyah - Rizeigat Baggara,5276,5276,...,07/22/2017 00:00:00,07/22/2017 00:00:00,2,25,0,0,27,27,27,POINT (26.091407 11.661619)
67,SUD-2021-1-666-2.1,2021,True,Clear,1,309,309,Sudan: Government,666,666,...,01/31/2021 00:00:00,01/31/2021 00:00:00,0,0,0,7,7,7,7,POINT (24.17 13.26)
69,SUD-2021-1-666-2.2,2021,True,Clear,1,309,309,Sudan: Government,666,666,...,01/24/2021 00:00:00,01/24/2021 00:00:00,0,0,0,7,7,7,7,POINT (24.17 13.26)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,SUD-2015-3-625-35,2015,True,Clear,3,112,480,Government of Sudan - Civilians,112,947,...,01/01/2015 00:00:00,01/04/2015 00:00:00,0,0,9,0,9,9,9,POINT (24.647945 13.287452)
1664,SUD-2020-1-666-5,2020,True,Clear,1,309,309,Sudan: Government,666,666,...,06/01/2020 00:00:00,06/01/2020 00:00:00,27,9,0,0,36,36,36,POINT (24.131 12.992)
1665,SUD-2020-2-16002-0,2020,True,Clear,2,16002,14695,SLM/A - MA - SLM/A - SB,16002,16002,...,05/21/2020 00:00:00,06/11/2020 00:00:00,0,0,0,17,17,17,17,POINT (24.327839 13.178889)
1667,CHA-2020-2-15074-2,2020,True,Clear,2,15074,13922,Fulani - Masalit,15074,15074,...,07/23/2020 00:00:00,07/26/2020 00:00:00,0,0,0,18,18,18,18,POINT (25.01174 11.36555)


In [36]:
# data = acled_df
# X = data.drop('FATALITIES', axis=1)
# y = data['FATALITIES']

data = ucdp_df
X = data.drop('best_est', axis=1)
y = data['best_est']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define categorical and numerical features
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create transformers for numeric and categorical features
## could use Imputer() from sklearn.preprocessing
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer to apply different preprocessing to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with the preprocessor and a linear regression classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LinearRegression())
])

# Train the model
model.fit(X_train, y_train)

# Evaluate the model (you can use appropriate metrics for your problem)
print("Model score: %.3f" % model.score(X_test, y_test))


Model score: 0.965
